In [1]:
import matplotlib.pyplot as plt
from skopt.plots import plot_gaussian_process
from skopt import gp_minimize



import pymc as pm
import dill as pickle
import pandas as pd
import seaborn as sns
import copy
import numpy as np
import arviz as az
from pyDOE import lhs, fullfact

from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import train_test_split


import params_team as params
import teams.teams_helpers as team_helpers
import analyze_study_3_data as asd

/home/sureshkj/anaconda3/envs/aug_taxi/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:246: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),


In [14]:
def simulate_objective(u, delta_c, delta_i):
    
    u = np.array(u)
    delta_c = np.array(delta_c)
    delta_i = np.array(delta_i)

    mdp_domain = 'augmented_taxi2'
    viz_flag = True
    learner_update_type = 'no_noise'

    # sim params
    max_learning_factor = params.max_learning_factor
    # initial_learning_factor = copy.deepcopy(learning_params['initial_learning_factor'])
    # learning_factor_delta = copy.deepcopy(learning_params['learning_factor_delta'])

    # initial_learning_factor = [learning_params[0]]
    # learning_factor_delta = [learning_params[1]/2, learning_params[1]]

    # learning_factor_delta = [learning_factor_delta_incorrect/2, learning_factor_delta_incorrect]

    # learning_factor_delta = [learning_factor_delta_correct, learning_factor_delta_incorrect]

    print('initial_learning_factor:', u, 'learning_factor_delta:', [delta_c, delta_i])

    # initialize (simulated) learner particle filters
    initial_learner_pf = copy.deepcopy(all_learner_pf['p1'])

    # prior interaction data
    prior_test_constraints = [np.array([[ 1,  0, -4]]), np.array([[-1,  0,  2]])]  # constraints of the first concept/KC
    all_test_constraints = {  1: [np.array([[ 1,  0, -4]]), np.array([[-1,  0,  2]])], \
                                2: [np.array([[0, 1, 2]]), np.array([[ 0, -1, -4]])], \
                                3: [np.array([[1, 1, 0]])]}
    min_BEC_constraints = [np.array([[1, 1, 0]]), np.array([[-1,  0,  2]]), np.array([[ 0, -1, -4]])]

    
    # initialize dataframes to save probability data
    simulated_interaction_data = pd.DataFrame()


    # simulate teaching loop
    prev_kc_id = 1
    demo_id = 1
    unit_constraints = [np.array([[0, 0, -1]])]
    objective = 0
    # objective = []

    learning_factor = u

    # plotting
    color_dict = {'demo': 'blue', 'remedial demo': 'purple', 'diagnostic test': 'red',  'remedial test': 'pink', 'diagnostic feedback': 'yellow', 'remedial feedback': 'orange', 'final test': 'green'}
    


    for user_id, user_interaction_data in prepared_interaction_data.iterrows():

        vars_filename_prefix = 'param_fit_user_' + str(user_id) + 'init_lf_' + str(u) + 'lf_delta_' + str(delta_c) + '_' + str(delta_i)
        # vars_filename_prefix = 'param_fit_user_' + str(user_id) + 'init_lf_' + str(initial_learning_factor) + 'lf_delta_' + str(learning_factor_delta)
        N_final_tests_correct = 0

        # print('user_id:', user_id, '.Len user_data:', len(user_interaction_data['kc_id']))

        # params to plot
        user_interaction_type = []
        user_learning_factor = []
        user_prob_BEC = []
        user_prob_KC = []
        user_loop_id = []
        concept_end_id = []
        plot_id = 0
        for loop_id in range(len(user_interaction_data['kc_id'])):
            
            # print('user_data_kcid:', user_data['kc_id'])
            current_kc_id = user_interaction_data['kc_id'][loop_id]
            current_interaction_type = user_interaction_data['interaction_types'][loop_id]
            # is_opt_response = user_data['is_opt_response'][loop_id]
            current_interaction_constraints = user_interaction_data['interaction_constraints'][loop_id]
            current_test_constraints = user_interaction_data['test_constraints'][loop_id]   

            # Not needed for now!
            # if current_interaction_type != 'final test' and current_kc_id <= max_kc:
            #     test_constraints = all_test_constraints[current_kc_id]
            # else:
            #     test_constraints = min_BEC_constraints

            if current_kc_id > prev_kc_id:
                # learning_factor = copy.deepcopy([initial_learning_factor])
                learning_factor = u
                # print('New KC! Resetting learning factor to initial value: ', learning_factor)
                concept_end_id.append(plot_id-1)

            # updates for various interaction types
            # Prior
            if current_interaction_type == 'prior':
                learner_pf = copy.deepcopy(initial_learner_pf)
            
            # Demo
            if current_interaction_type == 'demo':
                learner_pf.update(current_interaction_constraints, learning_factor, plot_title = 'Learner belief after demo. Interaction ID:  ' + str(loop_id) + ' for KC: ' + str(current_kc_id), viz_flag = viz_flag, vars_filename=vars_filename_prefix, model_type = learner_update_type)


            # Diagnostic Test
            if current_interaction_type == 'diagnostic test':
                # Nothing changes
                response_type = user_interaction_data['test_response_type'][loop_id][0]


            # Diagnostic Feedback
            if current_interaction_type == 'diagnostic feedback':
                # print('response_type: ', response_type)
                # if response_type == 'correct':
                #     learning_factor[0] = min(learning_factor[0] + learning_factor_delta[0], max_learning_factor)
                # elif response_type == 'incorrect':
                #     learning_factor[0] = min(learning_factor[0] + learning_factor_delta[1], max_learning_factor)
                # else:
                #     RuntimeError('Invalid response type')

                if response_type == 'correct':
                    learning_factor = min(learning_factor + delta_c, max_learning_factor)
                elif response_type == 'incorrect':
                    learning_factor = min(learning_factor + delta_i, max_learning_factor)
                else:
                    RuntimeError('Invalid response type')

                # updated learner model with corrective feedback
                plot_title =  ' Learner after corrective feedback for KC ' + str(current_kc_id)
                learner_pf.update(current_interaction_constraints, learning_factor, plot_title = plot_title, viz_flag = viz_flag, vars_filename = vars_filename_prefix, model_type = learner_update_type)

            # Remedial Demo
            if current_interaction_type == 'remedial demo':
                plot_title =  'Learner belief after remedial demo. Interaction ID: ' + str(loop_id) + ' for KC ' + str(current_kc_id)
                learner_pf.update(current_interaction_constraints, learning_factor, plot_title = plot_title, viz_flag = viz_flag, vars_filename=vars_filename_prefix, model_type = learner_update_type)
                
            # Remedial Test
            if current_interaction_type == 'remedial test':
                response_type = user_interaction_data['test_response_type'][loop_id][0]

            # Remedial Feedback
            if current_interaction_type == 'remedial feedback':
                # print('response_type: ', response_type)
                if response_type == 'correct':
                    learning_factor = min(learning_factor + delta_c, max_learning_factor)
                elif response_type == 'incorrect':
                    learning_factor = min(learning_factor + delta_i, max_learning_factor)
                else:
                    RuntimeError('Invalid response type')

                # updated learner model with corrective feedback
                plot_title =  ' Learner after remedial feedback for KC ' + str(current_kc_id)
                learner_pf.update(current_interaction_constraints, learning_factor, plot_title = plot_title, viz_flag = viz_flag, vars_filename = vars_filename_prefix, model_type = learner_update_type)

            # Final Test Performance
            if current_interaction_type == 'final test':
                if user_interaction_data['is_opt_response'][loop_id] == 1:
                    N_final_tests_correct += 1

            if 'prior_' not in current_interaction_type:
                # calculate probability of correct response
                learner_pf.calc_particles_probability(current_test_constraints)
                prop_particles_KC = learner_pf.particles_prob_correct


                learner_pf.calc_particles_probability(min_BEC_constraints)
                prop_particles_BEC = learner_pf.particles_prob_correct
                # print('loop_id: ', loop_id, 'interaction: ', current_interaction_type, 'prop_particles_BEC: ', prop_particles_BEC)

                # update loop vars
                user_interaction_type.append(current_interaction_type)
                user_learning_factor.append(learning_factor)
                user_prob_BEC.append(prop_particles_BEC)
                user_prob_KC.append(prop_particles_KC)
                user_loop_id.append(plot_id)
                plot_id += 1

            # update loop kcid
            prev_kc_id = current_kc_id

        
        
        # user_plot_data = pd.DataFrame({'loop_id': user_loop_id, 'interaction_type': user_interaction_type, 'learning_factor': user_learning_factor, 'prob_KC': user_prob_KC, 'prob_BEC': user_prob_BEC})
            
        # plt.figure(user_id)
        # ax0 = plt.gca()
        # print('user_id: ', user_id, 'user_plot_data:', user_plot_data)
        # sns.lineplot(data=user_plot_data, x='loop_id', y='prob_BEC', ax=ax0, color = 'blue').set(title='Obj. func. learning dynamics for user: ' + str(user_id))
        # sns.lineplot(data=user_plot_data, x='loop_id', y='prob_KC', ax=ax0, color = 'brown')
        # sns.lineplot(data=user_plot_data, x='loop_id', y='learning_factor', ax=ax0, color = 'green')

            
        # for id, row in user_plot_data.iterrows():
        #     print('id:', id)
        #     if row['interaction_type'] != 'prior':
        #         plt.axvspan(user_plot_data['loop_id'].iloc[id-1], row['loop_id'], alpha=0.2, color=color_dict[row['interaction_type']])
        #         plt.text(row['loop_id']-0.5, 0.3, row['interaction_type'], rotation=90, fontsize=12, weight="bold")

        # for id in concept_end_id:
        #     plt.axvline(x=id, color='black', linestyle='--', linewidth=2)
        
        # plt.show()

        # calculate final probability
        learner_pf.calc_particles_probability(min_BEC_constraints)
        prop_particles_BEC = learner_pf.particles_prob_correct

        # final test performance
        test_perf = N_final_tests_correct/6

        # update objective function
        objective += np.abs(prop_particles_BEC - test_perf)
        # objective.append(prop_particles_BEC)

        # print('user_id: ', user_id, 'N_final_tests_correct: ', N_final_tests_correct, 'prop_particles_BEC: ', prop_particles_BEC, 'test_perf: ', test_perf, 'objective: ', prop_particles_BEC - test_perf)

        

    return objective/len(prepared_interaction_data)

In [3]:
def load_abc_data(learner_type = 'low'):

    domain = 'at'
    filename = 'data/simulation/sim_experiments/parameter_estimation/abc_data_' + learner_type + '_domain_' + domain + '.pickle'

    try:
        # load data
        with open(filename, 'rb') as f:
            [prepared_interaction_data, interaction_output] = pickle.load(f)

    except:
        # prepare train test data
        with open ('data/prepared_interaction_data.pickle', 'rb') as f:
            all_interaction_data = pickle.load(f)

        with open('data/user_data_w_flag.pickle', 'rb') as f:
            all_user_data = pickle.load(f)

        
        if learner_type == 'test':
            user_data = all_user_data[(all_user_data['mislabeled_flag'] == 0) & (all_user_data['loop_condition'] != 'wt') & (all_user_data['loop_condition'] != 'wtcl') & \
                                ((all_user_data['N_final_correct_at'] == 2))]
        elif learner_type == 'low':
            user_data = all_user_data[(all_user_data['mislabeled_flag'] == 0) & (all_user_data['loop_condition'] != 'wt') & (all_user_data['loop_condition'] != 'wtcl') & \
                            ((all_user_data['N_final_correct_at'] == 2) | (all_user_data['N_final_correct_at'] == 3) | (all_user_data['N_final_correct_at'] == 4))]
        elif learner_type == 'high':
            user_data = all_user_data[(all_user_data['mislabeled_flag'] == 0) & (all_user_data['loop_condition'] != 'wt') & (all_user_data['loop_condition'] != 'wtcl') & \
                            ((all_user_data['N_final_correct_at'] == 5) | (all_user_data['N_final_correct_at'] == 6))]
        
        # input and output data
        unique_user_ids = user_data['user_id'].unique()
        prepared_interaction_data = pd.DataFrame()

        for user_id in unique_user_ids:
            prepared_interaction_data = prepared_interaction_data.append(all_interaction_data[all_interaction_data['user_id'] == user_id], ignore_index=True)

        interaction_output = user_data['N_final_correct_at']/6

        with open(filename, 'wb') as f:
            pickle.dump([prepared_interaction_data, interaction_output], f)

    return prepared_interaction_data, interaction_output

In [4]:
params.team_size = 1
params.max_learning_factor = 1.0


all_learner_pf = team_helpers.sample_team_pf(params.team_size, params.BEC['n_particles'], params.weights['val'], params.step_cost_flag, team_learning_factor = [0.8], team_prior = params.team_prior, pf_flag='learner', model_type = 'no_noise')


Updating particles for  p1 with prior knowledge in  no_noise  condition...


/home/sureshkj/anaconda3/envs/aug_taxi/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:178: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)


In [5]:
learner_type = 'test'

prepared_interaction_data, interaction_output = load_abc_data(learner_type = learner_type)

In [13]:
with pm.Model() as model_lv:
    u = pm.Normal("u", sigma=1)
    delta_c = pm.Normal("delta_c", sigma=1)
    delta_i = pm.Normal("delta_i", sigma=1)

    sim = pm.Simulator("sim", simulate_objective, params=(u, delta_c, delta_i), epsilon=10, observed=interaction_output)

    idata_lv = pm.sample_smc()

Initializing SMC sampler...
Sampling 12 chains in 12 jobs


TypeError: simulate_objective() takes 4 positional arguments but 5 were given
Apply node that caused the error: Simulator_sim_rv{0, (0, 0, 0), floatX, True}(simulator_rng, [8], 11, Reshape{0}.0, Reshape{0}.0, Reshape{0}.0)
Toposort index: 6
Inputs types: [RandomGeneratorType, TensorType(int64, shape=(1,)), TensorType(int64, shape=()), TensorType(float64, shape=()), TensorType(float64, shape=()), TensorType(float64, shape=())]
Inputs shapes: ['No shapes', (1,), (), (), (), ()]
Inputs strides: ['No strides', (8,), (), (), (), ()]
Inputs values: [Generator(PCG64) at 0x7FAC8F99D120, array([8]), array(11), array(1.84562648), array(1.6394472), array(0.05645382)]
Outputs clients: [['output'], [Composite{sqr((i2 * (i0 - i1)))}(sim{[0.3333333 ... .33333333]}, simulator_value, [0.1])]]

HINT: Re-running with most PyTensor optimizations disabled could provide a back-trace showing when this node was created. This can be done by setting the PyTensor flag 'optimizer=fast_compile'. If that does not w

In [15]:
basic_model = pm.Model()

with basic_model:

    # Priors for unknown model parameters
    u = pm.Normal('u', sigma=1)
    delta_c = pm.Normal('delta_c', sigma=1)
    delta_i = pm.Normal('delta_i', sigma=1)

    # Custom likelihood (negative because PyMC maximizes log likelihood)
    pm.Potential('likelihood', -simulate_objective(u, delta_c, delta_i))
    
    # draw 500 posterior samples
    trace = pm.sample(500)

initial_learning_factor: u learning_factor_delta: [delta_c, delta_i]


ValueError: setting an array element with a sequence.

error: Result from function call is not a proper array of floats.

<Figure size 432x288 with 0 Axes>